In [1]:
import socket
from docx import Document
import os
import csv
import xml.etree.ElementTree as ET
from reportlab.lib.pagesizes import letter
from reportlab.lib.units import inch


HOST = '127.0.0.1'  # Adresa IP a serverului
PORT = 65432        # Portul pe care serverul ascultă conexiuni

def receive_file():
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind((HOST, PORT))
        s.listen()
        print("Serverul este activ și ascultă conexiuni...")
        conn, addr = s.accept()
        with conn:
            print('Conexiune de la:', addr)
            print("Recepționare fișier...")
            with open('fisiere_server/received_file.docx', 'wb') as f:
                while True:
                    data = conn.recv(1024)
                    if not data:
                        break
                    f.write(data)
            print('Fișierul a fost primit cu succes.')

def read_docx(file_path):
    document = Document(file_path)
    text = ""
    for paragraph in document.paragraphs:
        text += paragraph.text + "\n"
    return text

def create_pdf(text, image_path, output_file):
    from reportlab.lib.pagesizes import letter
    from reportlab.lib.units import inch
    from reportlab.pdfgen import canvas

    # Facem un canvas
    c = canvas.Canvas(output_file, pagesize=letter)

    # Adaugam text din DOCX
    c.setFont("Helvetica", 12)
    text_lines = text.split("\n")
    y_position = inch * 10
    for line in text_lines:
        if y_position < inch:
            c.showPage()
            c.setFont("Helvetica", 12)
            y_position = inch * 10
        c.drawString(inch, y_position, line)
        y_position -= 12

    # Adaugam si imaginea 
    c.drawImage(image_path, inch, inch * 6, width=5*inch, height=3*inch)

    # Salvam Pdf-ul
    c.save()



def convert_image_to_pdf(image_path, pdf_path):
    from reportlab.lib.pagesizes import letter
    from reportlab.pdfgen import canvas
    from reportlab.lib.units import inch

    c = canvas.Canvas(pdf_path, pagesize=letter)
    c.drawImage(image_path, inch, inch * 6, width=5*inch, height=3*inch)
    c.save()

def generate_xml(data, output_path):
    try:
        root = ET.Element("data")
        for row in data:
            record = ET.SubElement(root, "record")
            for column in row:
                field = ET.SubElement(record, "field")
                field.text = str(column)
        
        tree = ET.ElementTree(root)
        tree.write(output_path)
        print(f"Fisierul XML a fost generat cu succes: {output_path}")
    except Exception as e:
        print(f"Eroare la generarea fisierului XML: {e}")

def read_csv(file_path):
    try:
        with open(file_path, 'r', newline='') as file:
            csv_reader = csv.reader(file)
            data = list(csv_reader)
            headers = data[0]
            rows = data[1:]
            
            for row in rows:
                for i in range(len(row)):
                    try:
                        row[i] = int(row[i])
                    except ValueError:
                        pass

            return headers, rows
    except Exception as e:
        print(f"Eroare la citirea fisierului CSV: {e}")
        return None, None

def main():
    receive_file()
    
    # Citirea continutului fisierului DOCX
    docx_file_path = "fisiere_server/received_file.docx"
    docx_text = read_docx(docx_file_path)

    # Crearea fisierului PDF
    pdf_file_path = "fisiere_server/crime_rates_report.pdf"
    create_pdf(docx_text, "total_crime_rates_plot.png", pdf_file_path)

    # Generarea fisierului XML
    docx_content = read_docx(docx_file_path)
    headers, rows = read_csv("fisiere_client/date.csv")

    if headers and rows:
        xml_file_path = os.path.join("fisiere_server", "output.xml")
        generate_xml([headers] + rows, xml_file_path)

    print("Procesul de generare a fișierelor a fost finalizat.")

if __name__ == "__main__":
    main()


Serverul este activ și ascultă conexiuni...
Conexiune de la: ('127.0.0.1', 53004)
Recepționare fișier...
Fișierul a fost primit cu succes.
Fisierul XML a fost generat cu succes: fisiere_server\output.xml
Procesul de generare a fișierelor a fost finalizat.
